In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from datetime import datetime, timedelta


In [2]:
hdb_resale_df = pd.read_csv(r"/workspaces/probabilistic-property/data/raw/HDB_Resale_From_2000.csv")
hdb_resale_df

/tmp/ipykernel_23437/2342360245.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  hdb_resale_df = pd.read_csv(r"/workspaces/probabilistic-property/data/raw/HDB_Resale_From_2000.csv")


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
949716,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0
949717,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0
949718,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0
949719,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0


In [3]:
hdb_resale_df.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object

# Sanity check columns

In [4]:
hdb_resale_df["flat_type"].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION', 'MULTI GENERATION'], dtype=object)

In [5]:
hdb_resale_df["flat_model"].unique()

array(['Improved', 'New Generation', 'DBSS', 'Standard', 'Apartment',
       'Simplified', 'Model A', 'Premium Apartment', 'Adjoined flat',
       'Model A-Maisonette', 'Maisonette', 'Type S1', 'Type S2',
       'Model A2', 'Terrace', 'Improved-Maisonette', 'Premium Maisonette',
       'Multi Generation', 'Premium Apartment Loft', '2-room', '3Gen',
       'IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT'], dtype=object)

In [6]:
hdb_resale_df["storey_range"].unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [7]:
hdb_resale_df["block"].unique().size

2738

# Investigate missing values
- No resolution required - the remaining_lease column was not included in older versions of the HDB Resale data but can be easily recalculated

In [ ]:
hdb_resale_df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
remaining_lease        709050
resale_price                0
dtype: int64

In [ ]:
hdb_resale_df[hdb_resale_df["remaining_lease"].isnull()]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
203518,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,NaN,147000.0
203519,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,NaN,144000.0
203520,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,NaN,159000.0
203521,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,NaN,167000.0
203522,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,NaN,163000.0
...,...,...,...,...,...,...,...,...,...,...,...
912563,1999-12,YISHUN,EXECUTIVE,611,YISHUN ST 61,10 TO 12,142.0,APARTMENT,1987,NaN,456000.0
912564,1999-12,YISHUN,EXECUTIVE,324,YISHUN CTRL,01 TO 03,142.0,APARTMENT,1988,NaN,408000.0
912565,1999-12,YISHUN,EXECUTIVE,392,YISHUN AVE 6,07 TO 09,146.0,MAISONETTE,1988,NaN,469000.0
912566,1999-12,YISHUN,EXECUTIVE,356,YISHUN RING RD,04 TO 06,146.0,MAISONETTE,1988,NaN,440000.0


# Engineer date columns

In [8]:
hdb_resale_df["date"] = pd.to_datetime(hdb_resale_df["month"])
hdb_resale_df["year"] = hdb_resale_df["date"].dt.year
hdb_resale_df["month"] = hdb_resale_df["date"].dt.month

In [9]:
# Validate the to_datetime operation
hdb_resale_df["date"].describe()

# datetime conversion looks fine - no extreme values and no errors thrown

count                           949721
mean     2007-01-23 02:49:37.634905856
min                1990-01-01 00:00:00
25%                1999-02-01 00:00:00
50%                2005-09-01 00:00:00
75%                2015-03-01 00:00:00
max                2025-04-01 00:00:00
Name: date, dtype: object

In [10]:
hdb_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,date,year
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017-01-01,2017
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017-01-01,2017
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017-01-01,2017
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017-01-01,2017
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017-01-01,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
949716,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0,2016-12-01,2016
949717,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0,2016-12-01,2016
949718,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0,2016-12-01,2016
949719,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0,2016-12-01,2016


# Engineer storey_range
- Instead of a string like "01 TO 03", convert to numerical or categorical?


In [11]:
# Get lower and upper bound of each existing category

hdb_resale_df["storey_lb"] = hdb_resale_df["storey_range"].str.split(" ").str[0].astype("int")
hdb_resale_df["storey_ub"] = hdb_resale_df["storey_range"].str.split(" ").str[-1].astype("int")
hdb_resale_df["storey_avg_floor"] = np.floor((hdb_resale_df["storey_lb"] + hdb_resale_df["storey_ub"]) /2 )


In [12]:
# Create full address column
hdb_resale_df["full_address"] = hdb_resale_df["block"] + "_" + hdb_resale_df["street_name"]

# Find max storey height for each address
hdb_resale_df["address_max_storey"] = hdb_resale_df.groupby("full_address")["storey_ub"].transform("max")
hdb_resale_df



,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,date,year,storey_lb,storey_ub,storey_avg_floor,full_address,address_max_storey
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017-01-01,2017,10,12,11.0,406_ANG MO KIO AVE 10,12
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017-01-01,2017,1,3,2.0,108_ANG MO KIO AVE 4,12
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017-01-01,2017,1,3,2.0,602_ANG MO KIO AVE 5,12
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017-01-01,2017,4,6,5.0,465_ANG MO KIO AVE 10,15
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017-01-01,2017,1,3,2.0,601_ANG MO KIO AVE 5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949716,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0,2016-12-01,2016,13,15,14.0,297_YISHUN ST 20,15
949717,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0,2016-12-01,2016,1,3,2.0,838_YISHUN ST 81,12
949718,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0,2016-12-01,2016,10,12,11.0,664_YISHUN AVE 4,12
949719,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0,2016-12-01,2016,1,3,2.0,325_YISHUN CTRL,15


In [13]:
hdb_resale_df["storey_prop_wrt_max"] = hdb_resale_df["storey_avg_floor"] / hdb_resale_df["address_max_storey"]
hdb_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,date,year,storey_lb,storey_ub,storey_avg_floor,full_address,address_max_storey,storey_prop_wrt_max
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017-01-01,2017,10,12,11.0,406_ANG MO KIO AVE 10,12,0.916667
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017-01-01,2017,1,3,2.0,108_ANG MO KIO AVE 4,12,0.166667
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017-01-01,2017,1,3,2.0,602_ANG MO KIO AVE 5,12,0.166667
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017-01-01,2017,4,6,5.0,465_ANG MO KIO AVE 10,15,0.333333
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017-01-01,2017,1,3,2.0,601_ANG MO KIO AVE 5,12,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949716,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0,2016-12-01,2016,13,15,14.0,297_YISHUN ST 20,15,0.933333
949717,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0,2016-12-01,2016,1,3,2.0,838_YISHUN ST 81,12,0.166667
949718,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0,2016-12-01,2016,10,12,11.0,664_YISHUN AVE 4,12,0.916667
949719,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0,2016-12-01,2016,1,3,2.0,325_YISHUN CTRL,15,0.133333


# Standardize namings for flat_type and flat_model

## flat_type
- flat_type has both: “MULTI-GENERATIONAL” and “MULTIGENERATIONAL”

In [14]:
hdb_resale_df["flat_type"].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION', 'MULTI GENERATION'], dtype=object)

In [15]:
hdb_resale_df["flat_type"] = hdb_resale_df["flat_type"].str.replace("MULTI GENERATION", "MULTI-GENERATION")
hdb_resale_df["flat_type"].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

## flat_model
- flat_model has both: “Improved” and “IMPROVED”, among others. Can be fixed with upper

In [16]:
hdb_resale_df["flat_model"] = hdb_resale_df["flat_model"].str.upper()
hdb_resale_df["flat_model"].unique()

array(['IMPROVED', 'NEW GENERATION', 'DBSS', 'STANDARD', 'APARTMENT',
       'SIMPLIFIED', 'MODEL A', 'PREMIUM APARTMENT', 'ADJOINED FLAT',
       'MODEL A-MAISONETTE', 'MAISONETTE', 'TYPE S1', 'TYPE S2',
       'MODEL A2', 'TERRACE', 'IMPROVED-MAISONETTE', 'PREMIUM MAISONETTE',
       'MULTI GENERATION', 'PREMIUM APARTMENT LOFT', '2-ROOM', '3GEN'],
      dtype=object)

# Recalculate remaining lease in years

In [17]:
hdb_resale_df["lease_commence_date"].unique()

array([1979, 1978, 1980, 1981, 1976, 1977, 2011, 2012, 1996, 1988, 1985,
       1986, 1974, 1984, 1983, 1987, 1982, 2000, 2001, 2005, 1989, 2010,
       1972, 1993, 1973, 1992, 1990, 1998, 2004, 1997, 1971, 1975, 1970,
       1969, 2013, 2008, 1999, 2003, 2002, 1995, 2006, 1967, 1968, 2007,
       1991, 1966, 2009, 1994, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021])

In [18]:
hdb_resale_df["lease_commence_date"] = pd.to_datetime(hdb_resale_df["lease_commence_date"], format = "%Y")

In [ ]:
hdb_resale_df["remaining_lease"] = hdb_resale_df["date"] - hdb_resale_df["lease_commence_date"]
hdb_resale_df["remaining_lease"] = hdb_resale_df["remaining_lease"] / timedelta(days = 365)
hdb_resale_df["remaining_lease"] = 99 - hdb_resale_df["remaining_lease"]

hdb_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,date,year,storey_lb,storey_ub,storey_avg_floor,full_address,address_max_storey,storey_prop_wrt_max
0,1,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,IMPROVED,1979-01-01,60.972603,232000.0,2017-01-01,2017,10,12,11.0,406_ANG MO KIO AVE 10,12,0.916667
1,1,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,NEW GENERATION,1978-01-01,59.972603,250000.0,2017-01-01,2017,1,3,2.0,108_ANG MO KIO AVE 4,12,0.166667
2,1,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,NEW GENERATION,1980-01-01,61.972603,262000.0,2017-01-01,2017,1,3,2.0,602_ANG MO KIO AVE 5,12,0.166667
3,1,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,NEW GENERATION,1980-01-01,61.972603,265000.0,2017-01-01,2017,4,6,5.0,465_ANG MO KIO AVE 10,15,0.333333
4,1,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,NEW GENERATION,1980-01-01,61.972603,265000.0,2017-01-01,2017,1,3,2.0,601_ANG MO KIO AVE 5,12,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949716,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,IMPROVED,2000-01-01,82.071233,488000.0,2016-12-01,2016,13,15,14.0,297_YISHUN ST 20,15,0.933333
949717,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,IMPROVED,1987-01-01,69.063014,455000.0,2016-12-01,2016,1,3,2.0,838_YISHUN ST 81,12,0.166667
949718,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,APARTMENT,1992-01-01,74.065753,778000.0,2016-12-01,2016,10,12,11.0,664_YISHUN AVE 4,12,0.916667
949719,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,MAISONETTE,1988-01-01,70.063014,575000.0,2016-12-01,2016,1,3,2.0,325_YISHUN CTRL,15,0.133333


# Per sqft pricing

## Normalize prices by per sqft + remaining lease

# Adjust prices for inflation